In [1]:
import numpy as np
import pandas as pd
from datascience import *
import csv

from string import punctuation, digits
from collections import Counter

from sklearn.feature_extraction.text import CountVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.feature_extraction.stop_words import ENGLISH_STOP_WORDS
stop_words = list(ENGLISH_STOP_WORDS)

import spacy
nlp = spacy.load('en')

import bokeh
from bokeh.plotting import figure, output_notebook, show
from bokeh.models import Range1d, BoxAnnotation, Label, Span, Legend



    Only loading the 'en' tokenizer.



## Greek Tragedies in 5th century BCE

This notebook contains 31 5th century Greek plays written by Sophocles, Aeschylus, and Euripidies. This data was created and gathered from http://classics.mit.edu.

### Loading Metadata

The following cell loads three different tables containing the metadata of the three writers and their plays in alphabetical order.

In [2]:
metadata_sophocles = Table().read_table("metadata_sophocles.csv")
metadata_aeschylus = Table().read_table("metadata_aeschylus.csv")
metadata_euripides = Table().read_table("metadata_euripides.csv")

In [3]:
metadata_sophocles

Title,Author,Date
Ajax,Sophocles,440
Antigone,Sophocles,442
Electra,Sophocles,410
Philoctetes,Sophocles,409
The Trachiniae,Sophocles,430


### Creating the Document Term Matrix through NLP

This cell reads and imports the corpus of Sophocles plays. The plays are split up through NLP by every instance of "THEEND", and then tokenized to create a word bag. The plays were entered into the text file alphabetically and correspond to the order that they appear in the metadata tables.

In [4]:
with open("sophocles_text.txt") as s:
    sophocles = s.read()

sophocles_list = sophocles.split("THEEND")

cv = CountVectorizer(stop_words = stop_words)
dtm = cv.fit_transform(sophocles_list)
tt = TfidfTransformer(norm='l1',use_idf=False)
dtm_tf = tt.fit_transform(dtm)

This cell creates a word list from the tokens and creates an array of said word list.

In [6]:
word_list = cv.get_feature_names()
dtm_array = dtm_tf.toarray()

This cell creates a document term matrix of every word that appears in each play.

In [7]:
sophocles_table_array = Table(word_list).with_rows(dtm_array)
sophocles_table_array

abandoned   | abandoning  | abate       | abhor       | abhors      | abide       | abides      | abidest     | abiding     | abject      | ablution    | abode       | abound      | abroad      | absent      | absolve     | absorbed    | abundance  | accents    | accept      | accepted    | accompanied | accomplished | accord      | according   | accost      | account     | accounted   | accursed    | accurst     | accuse      | accuser     | accusing    | accustomed  | achaean     | achelous    | acheron     | achieve     | achieved    | achilles   | aching      | acknowledge | acquire     | acquittal  | act         | acted       | acting      | action      | actions     | active      | acts        | add         | added       | adding      | address     | adieu       | adjudge     | admire      | admired     | admits      | admonitions | adoration   | adoring     | adorned     | advanced    | advances    | advancing   | adversaries | adversary   | adverse     | advice      | advise      | advised     | aeacid      | aegean      | aegisthus  | aenian      | aesculapius | aetolia     | afar        | afeard      | afflict     | afflicted   | affliction  | afflicts    | afford      | affording   | afield      | afraid      | afterward   | agamemnon   | age         | aged        | ages        | aggravated  | aggrieved   | aghast      | agitation   | ago         | agonies     | agony       | agora       | agree       | ah          | aiai        | aid         | aided       | aiding      | ailinon     | aim         | aimed       | aimless     | aims        | air         | airs        | ajax        | alarmed     | alarms      | alas       | albeit      | alcides    | alcmena     | alert      | alien       | aliens     | alight      | alike       | alive       | allege      | allegiance | alliance    | allies      | allow       | allowing    | allows      | ally        | almighty    | aloud       | altar       | altars      | altogether  | alway       | amain       | amazed     | ambush      | amends      | amid        | amidst      | amiss       | amphiaraus  | amphion    | ample       | ancestors   | ancestral   | ancient     | anew        | anger       | angered    | angry       | anguish     | ankle       | announce    | announced   | announcer   | anointed    | anointing   | answer      | answered    | answering   | antigone  | antilochus  | antistrophe | antisystema | antlered    | anxious     | apart       | aphrodite  | apollo      | appal       | appalling   | appeal     | appear      | appeared    | appears     | appease     | appetite    | applied     | apply       | appoint    | appointed   | appoints    | apprehend   | approach    | approaches  | approaching | approval    | approve     | approved   | aptly       | archer     | archers    | archery     | ares        | argive      | argives     | argos       | aright      | arise       | arisen     | arm         | armament    | armed       | arming      | armour      | arms        | army        | arose       | arouse      | aroused     | arraigns   | arrival     | arrive      | arrived     | arrogance   | arrow       | arrows      | art        | artemis     | arteries    | artifice    | artificer   | arts        | ascend      | ashamed     | ashes       | aside       | ask         | asked       | asking      | asks        | asleep      | aspect      | assail      | assailed    | assassin    | assault     | assaulting  | assembled   | assenting   | assist      | assistance  | associate   | assumed     | assured     | assuredly   | astonished  | astray     | asunder     | athena    | athenian    | athens      | athirst    | atonement   | atreidae    | atreides    | atreus      | attack      | attempt     | attempting  | attend      | attendant   | attendants  | attended    | attends     | attentive   | attest     | attestation | aught       | augur      | augury     | aulis       | author      | authority   | avail       | avails      | avenge      | avenger     | avengers    

This cell combines the metadata of each play with selected words from the document term matrix. These words were picked because of their relation to notions of gender, religion, the state and city, and war.

In [8]:
sophocles_table = metadata_sophocles
sophocles_frequencies = sophocles_table.with_column("men", sophocles_table_array["men"],
                                                    "women", sophocles_table_array["women"],
                                                    "son", sophocles_table_array["son"],
                                                    "daughter", sophocles_table_array["daughter"],
                                                    "father", sophocles_table_array["father"],
                                                    "mother", sophocles_table_array["mother"], 
                                                    "friends", sophocles_table_array["friends"], 
                                                    "fair", sophocles_table_array["fair"],
                                                    "joy", sophocles_table_array["joy"],
                                                    "love", sophocles_table_array["love"],
                                                    "rich", sophocles_table_array["rich"],
                                                    "god", sophocles_table_array["god"],
                                                    "time", sophocles_table_array["time"],
                                                    "dear", sophocles_table_array["dear"],
                                                    "soul", sophocles_table_array["soul"],
                                                    "mortal", sophocles_table_array["mortal"],
                                                    "power", sophocles_table_array["power"],
                                                    "pray", sophocles_table_array["pray"],
                                                    "fate", sophocles_table_array["fate"],
                                                    "city", sophocles_table_array["city"],
                                                    "house", sophocles_table_array["house"],
                                                    "home", sophocles_table_array["home"],                                                    
                                                    "soul", sophocles_table_array["soul"],
                                                    "woe", sophocles_table_array["woe"],
                                                    "dead", sophocles_table_array["dead"],                                                    
                                                    "dire", sophocles_table_array["dire"], 
                                                    "death", sophocles_table_array["death"],
                                                    "fear", sophocles_table_array["fear"],
                                                    "doom", sophocles_table_array["doom"],                                                    
                                                    "woe", sophocles_table_array["woe"],
                                                    "evil", sophocles_table_array["evil"],
                                                    "wretched", sophocles_table_array["wretched"],
                                                    "poor", sophocles_table_array["poor"],
                                                    "war", sophocles_table_array["war"]
                                                   )
sophocles_frequencies

Title,Author,Date,men,women,son,daughter,father,mother,friends,fair,joy,love,rich,god,time,dear,soul,mortal,power,pray,fate,city,house,home,woe,dead,dire,death,fear,doom,evil,wretched,poor,war
Ajax,Sophocles,440,0.00556328,0.000347705,0.00417246,0.000347705,0.00312935,0.00156467,0.00347705,0.000347705,0.00139082,0.00173853,0.000347705,0.00121697,0.00243394,0.00121697,0.00208623,0.00069541,0.000521558,0.000869263,0.00139082,0.000173853,0.000347705,0.00173853,0.00643255,0.00417246,0.000347705,0.00278164,0.000869263,0.00156467,0.00139082,0.000521558,0.000173853,0.000521558
Antigone,Sophocles,442,0.00440451,0.000704722,0.00458069,0.00123326,0.00352361,0.00246653,0.00123326,0.000880902,0.00158562,0.00193798,0,0.00158562,0.00123326,0.000880902,0.00158562,0.00123326,0.00211416,0.000704722,0.00246653,0.00387597,0.00352361,0.00105708,0.00264271,0.00475687,0.000704722,0.00493305,0.00123326,0.00387597,0.00264271,0.000704722,0.000352361,0.000528541
Electra,Sophocles,410,0.0022742,0.000812216,0.00162443,0.0022742,0.00698506,0.00519818,0.00389864,0.0011371,0.00211176,0.0011371,0.000324886,0.00178687,0.00211176,0.0011371,0.000974659,0.000812216,0.000812216,0.000974659,0.000649773,0,0.00568551,0.0022742,0.00324886,0.00438596,0.000974659,0.00276153,0.00178687,0.0011371,0.0011371,0.000649773,0.000649773,0
Philoctetes,Sophocles,409,0.00162075,0,0.0104448,0,0.00252116,0.000540249,0.00270124,0.000900414,0.000360166,0.000900414,0,0.000360166,0.00216099,0.00180083,0.00198091,0.000540249,0.0010805,0,0.00198091,0.000360166,0,0.000900414,0.00126058,0.00180083,0.000180083,0.00144066,0.00162075,0,0.000180083,0.00252116,0.00288133,0.000540249
The Trachiniae,Sophocles,430,0.00186463,0.000932314,0.00708559,0.0014917,0.00410218,0.00354279,0.00205109,0.00167817,0.00223755,0.00242402,0.000372926,0.00130524,0.00205109,0.00111878,0.00111878,0.000372926,0.000372926,0.00111878,0.000932314,0.00111878,0.00783144,0.00205109,0.00223755,0.00186463,0.000372926,0.0029834,0.00223755,0.0014917,0.00111878,0.00111878,0.000559388,0.000745851


## Analyzing the Document Term Matrix of each playwright

This process was performed on each playwright and the resulting tables were then exported to CSV files to keep this notebook neat. If one were to change the above steps but instead use Euripides or Aeschylus and create that document term matrix, they are capable of doing so with the files provided in the folder and would get the same tables that follow.

In [9]:
euripides = Table().read_table("euripides_frequencies.csv")
aeschylus = Table().read_table("aeschylus_frequencies.csv")
sophocles = Table().read_table("sophocles_frequencies.csv")

In [10]:
euripides

Title,Author,Date,men,women,son,daughter,father,mother,friends,fair,joy,love,rich,god,time,dear,soul,mortal,power,pray,fate,city,house,home,woe,dead,dire,death,fear,doom,evil,wretched,poor,war
Alcestis,Euripides,438,0.00452796,0.00475436,0.00611275,0.00158479,0.00543355,0.00498076,0.00203758,0.000226398,0.00113199,0.00294317,0.000905592,0.00226398,0.00249038,0.00271678,0.000452796,0.000679194,0.000679194,0.000452796,0.00339597,0.00135839,0.0135839,0.00158479,0.00226398,0.0126783,0,0.0101879,0.000905592,0.000679194,0.000452796,0.000226398,0,0
Andromache,Euripides,424,0.00443989,0.00187842,0.00802596,0.00631831,0.0039276,0.0039276,0.00290301,0.00068306,0.000170765,0.00153688,0.000512295,0.00426913,0.00119535,0.00102459,0.000512295,0.00034153,0.00102459,0.000853825,0.000853825,0.00290301,0.00665984,0.00358607,0.00324454,0.00102459,0.000170765,0.00495219,0.00153688,0.00034153,0.00273224,0.00034153,0.00187842,0.000512295
The Bacchantes,Euripides,410,0.00292826,0.00292826,0.00569383,0.00211485,0.00309094,0.00634456,0.000488043,0.00178949,0.00146413,0.000488043,0.000162681,0.0110623,0.00178949,0.00113877,0.000813405,0.00162681,0.000650724,0.000650724,0.000813405,0.00357898,0.00325362,0.00162681,0.00146413,0.000488043,0.000162681,0.000650724,0.000325362,0.000488043,0.000488043,0.000162681,0.000813405,0.000488043
The Cyclops,Euripides,408,0.00300933,0.000601866,0.0018056,0.000300933,0.0018056,0,0.00240746,0.00210653,0.000300933,0.00150466,0.000300933,0.00962985,0.000300933,0.00120373,0,0.000300933,0.000601866,0.00150466,0,0.000902799,0.000902799,0.00120373,0.000902799,0,0,0.000601866,0,0.000300933,0.000902799,0.000300933,0.000902799,0
Electra,Euripides,410,0.00177677,0.000969149,0.00371507,0.00355354,0.00985301,0.00856081,0.0032305,0.000807624,0.000807624,0.00177677,0.000484574,0.0025844,0.0019383,0.00161525,0.000807624,0.00032305,0.000646099,0.000646099,0.00113067,0.00306897,0.00646099,0.00371507,0.0012922,0.00145372,0,0.0032305,0.0019383,0.00113067,0.000646099,0.000807624,0.0025844,0
Hecuba,Euripides,424,0.0022783,0.00315457,0.00595864,0.00525762,0.0022783,0.00630915,0.00280407,0.00122678,0.000525762,0.000701016,0.000525762,0.0022783,0.00122678,0.00157729,0.000525762,0.000525762,0.00105152,0,0.00140203,0.00262881,0.00175254,0.00262881,0.00490712,0.00315457,0,0.00490712,0.000525762,0.000876271,0.00157729,0.000876271,0.00192779,0.000175254
Helen,Euripides,412,0.00250815,0.00100326,0.00263356,0.0032606,0.00225734,0.00213193,0.00225734,0.0016303,0.0016303,0.000627038,0.000250815,0.00188111,0.0016303,0.00100326,0.000877853,0.000376223,0.000376223,0.000752445,0.00112867,0.00050163,0.00275897,0.00288437,0.00388763,0.0050163,0.000125408,0.00351141,0.000376223,0.00050163,0.00150489,0.00050163,0.00100326,0.00050163
The Heracleidae,Euripides,429,0.00281996,0.000433839,0.0032538,0.00173536,0.00368764,0.00173536,0.0032538,0.0010846,0.000867679,0.000650759,0,0.00238612,0.00260304,0,0.00130152,0.000433839,0.00173536,0.000650759,0.000867679,0.00737527,0.00173536,0.0021692,0.000867679,0.000433839,0.00021692,0.00542299,0.00260304,0.000867679,0.00130152,0.00021692,0.000867679,0.000650759
Heracles,Euripides,416,0.00297946,0,0.00784068,0.000784068,0.00768386,0.0023522,0.00439078,0.00219539,0.00125451,0.00172495,0.000156814,0.00282264,0.00172495,0.00141132,0.000627254,0.000940881,0.000940881,0.000470441,0.00156814,0.00360671,0.00548847,0.000940881,0.00282264,0.00376353,0.000313627,0.00595892,0.000627254,0.000313627,0.00141132,0.000470441,0.00156814,0.000313627
Hippolytus,Euripides,428,0.00467727,0.00233863,0.00545681,0.00109136,0.00592454,0.00155909,0.00311818,0.000935454,0.000935454,0.00717181,0,0.00218272,0.00140318,0.00140318,0.00233863,0.00109136,0.000623636,0.000935454,0.001715,0.000623636,0.0053009,0.00218272,0.00483318,0.00218272,0.000155909,0.00420954,0.000779545,0.000155909,0.00280636,0.000311818,0.00280636,0.000155909


### Visualization

The graphs were made utilizing Bokeh open source data visualizations. https://bokeh.pydata.org

In [11]:
output_notebook()

Loading BokehJS ...

In [12]:
euripides_sort = euripides.sort("Date")
aeschylus_sort = aeschylus.sort("Date")
sophocles_sort = sophocles.sort("Date")

In [13]:
x = euripides.sort("Date").column("Date")

men = euripides_sort.column("men")
women = euripides_sort.column("women")
son = euripides_sort.column("son")
daughter = euripides_sort.column("daughter")
father = euripides_sort.column("father")
mother = euripides_sort.column("mother")
# friends = euripides_sort.column("friends")
fair = euripides_sort.column("fair")
joy = euripides_sort.column("joy")
love = euripides_sort.column("love")
# rich = euripides_sort.column("rich")
god = euripides_sort.column("god")
# time = euripides_sort.column("time")
# dear = euripides_sort.column("dear")
soul = euripides_sort.column("soul")
mortal = euripides_sort.column("mortal")
power = euripides_sort.column("power") 
pray = euripides_sort.column("pray")
fate = euripides_sort.column("fate")
city = euripides_sort.column("city")
house = euripides_sort.column("house")
# home = euripides_sort.column("home")
woe = euripides_sort.column("woe") 
# dead = euripides_sort.column("dead")
dire = euripides_sort.column("dire") 
death = euripides_sort.column("death")
fear = euripides_sort.column("fear") 
# doom = euripides_sort.column("doom")
evil = euripides_sort.column("evil") 
wretched = euripides_sort.column("wretched")
# poor = euripides_sort.column("poor") 
war = euripides_sort.column("war")

p = figure(plot_width = 850, plot_height = 425, toolbar_location="above")

p.xaxis.axis_label = "Year (BCE)"
p.xaxis.axis_label_text_font_size = "12pt"

p.yaxis.axis_label = "DTM Frequency"
p.yaxis.axis_label_text_font_size = "12pt"

# ranges
p.y_range = Range1d(0, .015)
p.x_range = Range1d(455, 400)

w1 = p.line(x, men, line_width = 3, color = "blue", alpha = 0.5, line_cap = "round", legend = "men")
w2 = p.line(x, women, line_width = 3, color = "hotpink", alpha = 0.5, line_cap = "round", legend = "women")
w3 = p.line(x, son, line_width = 3, color = "lightblue", alpha = 0.5, line_cap = "round", legend = "son")
w4 = p.line(x, daughter, line_width = 3, color = "pink", alpha = 0.5, line_cap = "round", legend = "daughter")
w5 = p.line(x, father, line_width = 3, color = "dodgerblue", alpha = 0.5, line_cap = "round", legend = "father")
w6 = p.line(x, mother, line_width = 3, color = "magenta", alpha = 0.5, line_cap = "round", legend = "magenta")
# w7 = p.line(x, friends, line_width = 3, color = "yellow", alpha = 0.5, line_cap = "round", legend = "friends")
w8 = p.line(x, fair, line_width = 3, color = "springgreen", alpha = 0.5, line_cap = "round", legend = "fair")
w9 = p.line(x, joy, line_width = 3, color = "lightsalmon", alpha = 0.5, line_cap = "round", legend = "joy")
w10 = p.line(x, love, line_width = 3, color = "mediumvioletred", alpha = 0.5, line_cap = "round", legend = "love")
# w11 = p.line(x, rich, line_width = 3, color = "gold", alpha = 0.5, line_cap = "round", legend = "rich")
w12 = p.line(x, god, line_width = 3, color = "black", alpha = 0.5, line_cap = "round", legend = "god")
# w13 = p.line(x, time, line_width = 3, color = "mediumvioletred", alpha = 0.5, line_cap = "round", legend = "time")
# w14 = p.line(x, dear, line_width = 3, color = "mediumvioletred", alpha = 0.5, line_cap = "round", legend = "dear")
w15 = p.line(x, soul, line_width = 3, color = "powderblue", alpha = 0.5, line_cap = "round", legend = "soul")
w16 = p.line(x, mortal, line_width = 3, color = "olivedrab", alpha = 0.5, line_cap = "round", legend = "mortal")
w17 = p.line(x, power, line_width = 3, color = "crimson", alpha = 0.5, line_cap = "round", legend = "power")
w18 = p.line(x, pray, line_width = 3, color = "darkturquoise", alpha = 0.5, line_cap = "round", legend = "pray")
w19 = p.line(x, fate, line_width = 3, color = "indigo", alpha = 0.5, line_cap = "round", legend = "fate")
w20 = p.line(x, city, line_width = 3, color = "darkgray", alpha = 0.5, line_cap = "round", legend = "city")
w21 = p.line(x, house, line_width = 3, color = "salmon", alpha = 0.5, line_cap = "round", legend = "house")
# w22 = p.line(x, home, line_width = 3, color = "mediumvioletred", alpha = 0.5, line_cap = "round", legend = "home")
w23 = p.line(x, woe, line_width = 3, color = "midnightblue", alpha = 0.5, line_cap = "round", legend = "woe")
# w24 = p.line(x, dead, line_width = 3, color = "mediumvioletred", alpha = 0.5, line_cap = "round", legend = "dead")
w25 = p.line(x, dire, line_width = 3, color = "chartreuse", alpha = 0.5, line_cap = "round", legend = "dire")
w26 = p.line(x, death, line_width = 3, color = "seashell", alpha = 0.5, line_cap = "round", legend = "death")
w27 = p.line(x, fear, line_width = 3, color = "brown", alpha = 0.5, line_cap = "round", legend = "fear")
# w28 = p.line(x, doom, line_width = 3, color = "mediumvioletred", alpha = 0.5, line_cap = "round", legend = "doom")
w29 = p.line(x, evil, line_width = 3, color = "khaki", alpha = 0.5, line_cap = "round", legend = "evil")
w30 = p.line(x, wretched, line_width = 3, color = "purple", alpha = 0.5, line_cap = "round", legend = "wretched")
# w31 = p.line(x, poor, line_width = 3, color = "mediumvioletred", alpha = 0.5, line_cap = "round", legend = "poor")
w32 = p.line(x, war, line_width = 3, color = "red", alpha = 0.5, line_cap = "round", legend = "war")


# legend
legend = Legend(items = [
    ("men", [w1]),
    ("women", [w2]),
    ("son", [w3]),
    ("daughter", [w4]),
    ("father", [w5]),
    ("mother", [w6]),
#    ("friends", [w7]),
    ("fair", [w8]),
    ("joy", [w9]),
    ("love", [w10]),
#    ("rich", [w11]),
    ("god", [w12]),
#    ("time", [w13]),
#    ("dear", [w14]),
    ("soul", [w15]),
    ("mortal", [w16]),
    ("power", [w17]),
    ("pray", [w18]),
    ("fate", [w19]),
    ("city", [w20]),
    ("house", [w21]),
#    ("home", [w22]),
    ("woe", [w23]),
#    ("dead", [w24]),
    ("dire", [w25]),
    ("death", [w26]),
    ("fear", [w27]),
#    ("doom", [w28]),
    ("evil", [w29]),
    ("wretched", [w30]),
#    ("poor", [w31]),
    ("war", [w32]),])
p.add_layout(legend, "right")
p.legend.location = (30, 5)
p.legend.click_policy = "hide"
p.legend.label_text_font_size = "8pt"
p.legend.spacing = -4
p.legend.label_standoff = 10
p.legend.padding = 30
p.legend.glyph_width = 4

# important stuff
war1 = BoxAnnotation(left = 431, right = 421, fill_color = "red", fill_alpha = 0.1)
war2 = BoxAnnotation(left = 411, right = 404, fill_color = "red", fill_alpha = 0.1)
p.add_layout(war1)
p.add_layout(war2)

# war3 = BoxAnnotation(left = 499, right = 449, fill_color = "red", fill_alpha = 0.1)
# p.add_layout(war3)

# wars and important dates
# archidamian_war = Label(x = 433.5, y = 0.009, text = "Peloponnesian War (Archidamian)", render_mode = "css", text_font_size = "10pt", background_fill_color = "white")
# second_war = Label(x = 415, y = 0.009, text = "Peloponnesian War (Second War)", render_mode = "css", text_font_size = "10pt", background_fill_color = "white")
# persian_war = Label(x = 475, y = 0.009, text = "Persian Wars", render_mode = "css", text_font_size = "10pt", background_fill_color = "white")
# p.add_layout(archidamian_war)
# p.add_layout(second_war)
# p.add_layout(persian_war)

# p.line(421, make_array(-0.1, 0.1), line_width = 1, color = "navy", line_dash = "dashed", line_cap = "round", legend = "Peace of Nicias", alpha = .75)
# p.line(415, make_array(-0.1, 0.1), line_width = 1, color = "blue", line_dash = "dashed", line_cap = "round", legend = "Sicilian Expedition", alpha = .75)

# gid rid of line
p.xgrid.grid_line_color = None

show(p)